# Home-Based Remote Patient Monitoring

This is a notebook version of the amazing article [Home Based Patient Monitoring - A Fast Track FHIR Story for the MIT Grand Hack 2022](https://community.intersystems.com/post/home-based-patient-monitoring-fast-track-fhir-story-mit-grand-hack-2022), by [Ron Sweeney](https://community.intersystems.com/user/ron-sweeney-0).

The article was created to guide people in the MIT Grand Hack 2022. It shows how to use FHIR API to register in InterSystems FHIR Server:

* an organization, for a hypothetical startup
* a patient managed by such startup
* a hypothetical clinical device and metrics generated by it
* how to query such metrics

## Setup

First, we import the lib `requests` to access the FHIR API, define the base URL to the server and some utility functions.

Here, is supposed that an INterSystems FHIR Server is listening in the localhost:52773 address, with no authentication.

In [1]:
import requests

In [2]:
# replace here with your server URL
base_url = "http://localhost:52773/fhir/r4"

In [3]:
# A simple method to peform POSTs to the FHIR Server
def post_resource(path, payload):
    headers = {"Content-Type": "application/fhir+json;charset=utf-8"}
    api_url = f"{base_url}/{path}"
    response = requests.post(api_url, headers=headers, json=payload)
    return response

In [4]:
# A simple method to peform GETs to the FHIR Server
def get_resource(path):
    api_url = f"{base_url}/{path}"
    response = requests.get(api_url)
    resource = response.json()
    return resource

In [5]:
# Function to get the ID created by FHIR Server. 
# For resource creation, the FHIR Server responds with a 
# HTTP 201 code and the created ID in the LOCALTION header.
def get_location_id(response):
    return response.headers["LOCATION"].split("/")[-3]

## Establish Your Startup Organization

Now, let's create an organization called 'What Up Start Up', and fill it with its information.

In [6]:
# An Organization FHIR resource definition. For more information, check https://build.fhir.org/organization.html
organization_payload = {
  "resourceType": "Organization",
  "name": "What Up Start Up",
  "alias": [
    "Grand Hack Start Up"
  ],
  "telecom": [
    {
      "system": "phone",
      "value": "(+1) 617-677-7777"
    },
    {
      "system": "fax",
      "value": "(+1) 617-677-6622"
    },
    {
      "system": "email",
      "value": "hq@whatupstartup.com"
    }
  ],
  "address": [
    {
      "line": [
        "100 One Memorial"
      ],
      "city": "Cambridge",
      "state": "MA",
      "postalCode": "02142",
      "country": "USA"
    }
  ],
}
r = post_resource("Organization", organization_payload)
r

<Response [201]>

In [7]:
# retrieve the ID created by the POST
created_id = get_location_id(r)
created_id

'33'

In [8]:
# check if we can get the resource recently created
get_resource(f"Organization/{created_id}")

{'resourceType': 'Organization',
 'name': 'What Up Start Up',
 'alias': ['Grand Hack Start Up'],
 'telecom': [{'system': 'phone', 'value': '(+1) 617-677-7777'},
  {'system': 'fax', 'value': '(+1) 617-677-6622'},
  {'system': 'email', 'value': 'hq@whatupstartup.com'}],
 'address': [{'line': ['100 One Memorial'],
   'city': 'Cambridge',
   'state': 'MA',
   'postalCode': '02142',
   'country': 'USA'}],
 'id': '33',
 'meta': {'lastUpdated': '2023-11-28T20:36:01Z', 'versionId': '1'}}

## Add Your Startup Device

The next step is to register a hypothetical clinical device. In the FHIR Server, such device will be identified by `Device/sd2000`.

In [9]:
# A Device FHIR resource definition. For more information, check https://build.fhir.org/device.html
device_payload = {
  "resourceType": "Device",
  "id": "smiledetector2000",
  "identifier": [
    {
      "system": "http://whatupstartup.com/devices/id",
      "value": "sd2000"
    }
  ],
}
r = post_resource("Device", device_payload)
r

<Response [201]>

In [10]:
# retrieve the ID created by the POST
created_id = get_location_id(r)
created_id

'34'

In [11]:
# check if we can get the resource recently created
get_resource(f"Device/{created_id}")

{'resourceType': 'Device',
 'id': '34',
 'identifier': [{'system': 'http://whatupstartup.com/devices/id',
   'value': 'sd2000'}],
 'meta': {'lastUpdated': '2023-11-28T20:36:01Z', 'versionId': '1'}}

## Establish Your Startup Device Metric

Now, it's necessary register a metric schema generated by the `Device/sd2000` device, identified by `DeviceMetric/smiles`.

Here information about what your device measure and how to interpret them could be regitered - iInformation like unit, calibration, read frequency/period, etc. For more informationm, check https://build.fhir.org/devicemetric.html.

The link to the device - in this case the device `Device/sd2000`, is done by this property:

```json
  "source": {
    "reference": "Device/sd2000"
  },
```

The device metric setting information are sent in the other properties.

In [12]:
# A Device FHIR resource definition. For more information, check https://build.fhir.org/devicemetric.html
device_metric_payload = {
  "resourceType": "DeviceMetric",
  "id": "smiles",
  "identifier": [
    {
      "system": "http://whatupstartup.com/devicemetric/id",
      "value": "smilesmetric"
    }
  ],
  "type": {
    "coding": [
      {
        "system": "urn:iso:std:iso:11073:10101",
        "code": "2000",
        "display": "SMILES"
      }
    ]
  },
  "unit": {
    "coding": [
      {
        "system": "urn:iso:std:iso:11073:10101",
        "code": "262688",
        "display": "SMILES_PER_SECOND"
      }
    ]
  },
  "source": {
    "reference": "Device/sd2000"
  },
  "operationalStatus": "on",
  "color": "blue",
  "category": "measurement",
  "measurementPeriod": {
    "repeat": {
      "frequency": 1,
      "period": 1,
      "periodUnit": "s"
    }
  },
  "calibration": [
    {
      "type": "two-point",
      "state": "calibrated",
      "time": "2021-12-28T09:03:04-05:00"
    }
  ],
}
r = post_resource("DeviceMetric", device_metric_payload)
r

<Response [201]>

In [13]:
# retrieve the ID created by the POST
created_id = get_location_id(r)
created_id

'35'

In [14]:
# check if we can get the resource recently created
get_resource(f"DeviceMetric/{created_id}")

{'resourceType': 'DeviceMetric',
 'id': '35',
 'identifier': [{'system': 'http://whatupstartup.com/devicemetric/id',
   'value': 'smilesmetric'}],
 'type': {'coding': [{'system': 'urn:iso:std:iso:11073:10101',
    'code': '2000',
    'display': 'SMILES'}]},
 'unit': {'coding': [{'system': 'urn:iso:std:iso:11073:10101',
    'code': '262688',
    'display': 'SMILES_PER_SECOND'}]},
 'source': {'reference': 'Device/sd2000'},
 'operationalStatus': 'on',
 'color': 'blue',
 'category': 'measurement',
 'measurementPeriod': {'repeat': {'frequency': 1,
   'period': 1,
   'periodUnit': 's'}},
 'calibration': [{'type': 'two-point',
   'state': 'calibrated',
   'time': '2021-12-28T09:03:04-05:00'}],
 'meta': {'lastUpdated': '2023-11-28T20:36:01Z', 'versionId': '1'}}

## Register a Patient in our Organization/Startup

A patient is created to get observations on metrics of the new device.

The patient is identified by `Patient/ourpatient`.

In [15]:
# A Patient FHIR resource definition. For more information, check https://build.fhir.org/patient.html
patient_payload = {
  "resourceType": "Patient",
  "id": "ourpatient",
  "identifier": [
    {
      "use": "usual",
      "type": {
        "coding": [
          {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "MR"
          }
        ]
      },
      "system": "urn:oid:0.1.2.3.4.5.6.7",
      "value": "1"
    }
  ],
  "active": True,
  "name": [
    {
      "use": "official",
      "family": "Everywhere",
      "given": [
        "Adam"
      ]
    }
  ],
  "gender": "male",
  "contact": [
    {
      "relationship": [
        {
          "coding": [
            {
              "system": "http://terminology.hl7.org/CodeSystem/v2-0131",
              "code": "E"
            }
          ]
        }
      ],
      "organization": {
        "reference": "Organization/wusu",
        "display": "What Up Start Up"
      }
    }
  ],
  "managingOrganization": {
    "reference": "Organization/wusu",
    "display": "What Up Start Up"
  },

}
r = post_resource("Patient", patient_payload)
r

<Response [201]>

In [16]:
# retrieve the ID created by the POST
created_id = get_location_id(r)
created_id

'36'

In [17]:
# check if we can get the resource recently created
get_resource(f"Patient/{created_id}")

{'resourceType': 'Patient',
 'id': '36',
 'identifier': [{'use': 'usual',
   'type': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v2-0203',
      'code': 'MR'}]},
   'system': 'urn:oid:0.1.2.3.4.5.6.7',
   'value': '1'}],
 'active': True,
 'name': [{'use': 'official', 'family': 'Everywhere', 'given': ['Adam']}],
 'gender': 'male',
 'contact': [{'relationship': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v2-0131',
       'code': 'E'}]}],
   'organization': {'reference': 'Organization/wusu',
    'display': 'What Up Start Up'}}],
 'managingOrganization': {'reference': 'Organization/wusu',
  'display': 'What Up Start Up'},
 'meta': {'lastUpdated': '2023-11-28T20:36:02Z', 'versionId': '1'}}

## Have Your Device Emit a Metric!

Now we have a device, a way to record metrics from such device and a patient, let's register an observation for them.

The link to the patient is done by this property:

```json
  "subject": {
    "reference": "Patient/ourpatient"
  },
```

And the link to the device:

```json
  "device": {
    "reference": "DeviceMetric/smiles"
  },
```

The observation values are registered in the other properties.

In [18]:
# A Device FHIR resource definition. For more information, check https://build.fhir.org/observation.html
observation_payload = {
  "resourceType": "Observation",
  "id": "smiles2000",
  "identifier": [
    {
      "system": "http://whatupstartup.com/observation/id",
      "value": "1111111"
    }
  ],
  "status": "final",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/observation-category",
          "code": "vital-signs",
          "display": "Vital Signs"
        }
      ],
      "text": "Vital Signs"
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "2708-6",
        "display": "Smiles per Second"
      },
      {
        "system": "http://loinc.org",
        "code": "59408-5",
        "display": "Smiles per Second our Device"
      },
      {
        "system": "urn:iso:std:iso:11073:10101",
        "code": "150456",
        "display": "SMILES"
      }
    ]
  },
  "subject": {
    "reference": "Patient/ourpatient"
  },
  "effectiveDateTime": "2014-12-05T09:30:10+01:00",
  "valueQuantity": {
    "value": 4,
    "unit": "s",
    "system": "http://unitsofmeasure.org",
    "code": "s"
  },
  "interpretation": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation",
          "code": "N",
          "display": "Normal"
        }
      ],
      "text": "Normal (applies to non-numeric results)"
    }
  ],
  "device": {
    "reference": "DeviceMetric/smiles"
  },
}
r = post_resource("Observation", observation_payload)
r

<Response [201]>

In [19]:
# retrieve the ID created by the POST
created_id = get_location_id(r)
created_id

'37'

In [20]:
# check if we can get the resource recently created
get_resource(f"Observation/{created_id}")

{'resourceType': 'Observation',
 'id': '37',
 'identifier': [{'system': 'http://whatupstartup.com/observation/id',
   'value': '1111111'}],
 'status': 'final',
 'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
     'code': 'vital-signs',
     'display': 'Vital Signs'}],
   'text': 'Vital Signs'}],
 'code': {'coding': [{'system': 'http://loinc.org',
    'code': '2708-6',
    'display': 'Smiles per Second'},
   {'system': 'http://loinc.org',
    'code': '59408-5',
    'display': 'Smiles per Second our Device'},
   {'system': 'urn:iso:std:iso:11073:10101',
    'code': '150456',
    'display': 'SMILES'}]},
 'subject': {'reference': 'Patient/ourpatient'},
 'effectiveDateTime': '2014-12-05T09:30:10+01:00',
 'valueQuantity': {'value': 4,
  'unit': 's',
  'system': 'http://unitsofmeasure.org',
  'code': 's'},
 'interpretation': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation',
     'code': 'N',
     'd

## Add Heart Rate Measure


For illustration, let's register an observation for a real metric - heart rate. In the [LOINC](https://en.wikipedia.org/wiki/LOINC) standard for medical laboratory observations identification, its code is [8867-4](https://loinc.org/8867-4).

This information is done by this property:

```json
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "8867-4",
        "display": "Heart rate"
      }
    ],
    "text": "Heart rate"
  },
```

In the observation for the hypothetical device, fake values were used:

```json
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "2708-6",
        "display": "Smiles per Second"
      },
      {
        "system": "http://loinc.org",
        "code": "59408-5",
        "display": "Smiles per Second our Device"
      },
      {
        "system": "urn:iso:std:iso:11073:10101",
        "code": "150456",
        "display": "SMILES"
      }
    ]
  },
```

In [21]:
# A Device FHIR resource definition. For more information, check https://build.fhir.org/observation.html
observation_payload = {
  "resourceType": "Observation",
  "id": "heart-rate",
  "meta": {
    "profile": [
      "http://hl7.org/fhir/StructureDefinition/vitalsigns"
    ]
  },
  "text": {
    "status": "generated",
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Generated Narrative with Details</b></p><p><b>id</b>: heart-rate</p><p><b>meta</b>: </p><p><b>status</b>: final</p><p><b>category</b>: Vital Signs <span>(Details : {http://terminology.hl7.org/CodeSystem/observation-category code 'vital-signs' = 'Vital Signs', given as 'Vital Signs'})</span></p><p><b>code</b>: Heart rate <span>(Details : {LOINC code '8867-4' = 'Heart rate', given as 'Heart rate'})</span></p><p><b>subject</b>: <a>Patient/ourpatient</a></p><p><b>effective</b>: 04/16/2022</p><p><b>value</b>: 44 beats/minute<span> (Details: UCUM code /min = '/min')</span></p></div>"
  },
  "status": "final",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/observation-category",
          "code": "vital-signs",
          "display": "Vital Signs"
        }
      ],
      "text": "Vital Signs"
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "8867-4",
        "display": "Heart rate"
      }
    ],
    "text": "Heart rate"
  },
  "subject": {
    "reference": "Patient/ourpatient"
  },
  "effectiveDateTime": "2022-04-16T09:30:10+01:00",
  "valueQuantity": {
    "value": 44,
    "unit": "beats/minute",
    "system": "http://unitsofmeasure.org",
    "code": "/min"
  }
}
r = post_resource("Observation", observation_payload)
r

<Response [201]>

In [22]:
# retrieve the ID created by the POST
created_id = get_location_id(r)
created_id

'38'

In [23]:
# check if we can get the resource recently created
get_resource(f"Observation/{created_id}")

{'resourceType': 'Observation',
 'id': '38',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/vitalsigns'],
  'lastUpdated': '2023-11-28T20:36:02Z',
  'versionId': '1'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><p><b>Generated Narrative with Details</b></p><p><b>id</b>: heart-rate</p><p><b>meta</b>: </p><p><b>status</b>: final</p><p><b>category</b>: Vital Signs <span>(Details : {http://terminology.hl7.org/CodeSystem/observation-category code \'vital-signs\' = \'Vital Signs\', given as \'Vital Signs\'})</span></p><p><b>code</b>: Heart rate <span>(Details : {LOINC code \'8867-4\' = \'Heart rate\', given as \'Heart rate\'})</span></p><p><b>subject</b>: <a>Patient/ourpatient</a></p><p><b>effective</b>: 04/16/2022</p><p><b>value</b>: 44 beats/minute<span> (Details: UCUM code /min = \'/min\')</span></p></div>'},
 'status': 'final',
 'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
     

## Get All Smiles For Patient Recorded by Our Device


Once observations were registered for the new device and the patient, we can query them.

Note that for observations, the link to the device is identified by its metric - `DeviceMetric/smiles` in this case.

In [24]:
# get the first 100 observations for the metric `DeviceMetrics/smiles` and patient `Patient/ourpatient`
get_resource(f"Observation?device=DeviceMetric%2Fsmiles&patient=Patient/ourpatient")

{'resourceType': 'Bundle',
 'id': '40a80b90-8e2f-11ee-b96a-0242ac180002',
 'type': 'searchset',
 'timestamp': '2023-11-28T20:36:02Z',
 'total': 2,
 'link': [{'relation': 'self',
   'url': 'http://localhost:52773/fhir/r4/Observation?device=DeviceMetric%2Fsmiles&patient=Patient%2Fourpatient'}],
 'entry': [{'fullUrl': 'http://localhost:52773/fhir/r4/Observation/31',
   'resource': {'resourceType': 'Observation',
    'id': '31',
    'identifier': [{'system': 'http://whatupstartup.com/observation/id',
      'value': '1111111'}],
    'status': 'final',
    'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
        'code': 'vital-signs',
        'display': 'Vital Signs'}],
      'text': 'Vital Signs'}],
    'code': {'coding': [{'system': 'http://loinc.org',
       'code': '2708-6',
       'display': 'Smiles per Second'},
      {'system': 'http://loinc.org',
       'code': '59408-5',
       'display': 'Smiles per Second our Device'},
      {'syste

Note that it's possible to query by other observation parameters.

For instance, another way to query the same resources could be done using the patient `ourpatient`, category `vital-sign` and code `150456`. The code was defined by this property in the observation data:

```json
  "code": {
    "coding": [
      ...,
      {
        "system": "urn:iso:std:iso:11073:10101",
        "code": "150456",
        "display": "SMILES"
      }
    ]
  },
```

For more information on how to query in the InterSystems FHIR Server, check https://docs.intersystems.com/irisforhealthlatest/csp/docbook/DocBook.UI.Page.cls?KEY=HXFHIR_server_arch_supported.

In [25]:
# get the first 100 observations for the patient `ourpatient`, category `vital-sign` and code `150456`
get_resource(f"Observation?patient=ourpatient&category=vital-signs&code=150456")

{'resourceType': 'Bundle',
 'id': '62437438-8e2f-11ee-b96a-0242ac180002',
 'type': 'searchset',
 'timestamp': '2023-11-28T20:36:02Z',
 'total': 2,
 'link': [{'relation': 'self',
   'url': 'http://localhost:52773/fhir/r4/Observation?category=vital-signs&code=150456&patient=ourpatient'}],
 'entry': [{'fullUrl': 'http://localhost:52773/fhir/r4/Observation/31',
   'resource': {'resourceType': 'Observation',
    'id': '31',
    'identifier': [{'system': 'http://whatupstartup.com/observation/id',
      'value': '1111111'}],
    'status': 'final',
    'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
        'code': 'vital-signs',
        'display': 'Vital Signs'}],
      'text': 'Vital Signs'}],
    'code': {'coding': [{'system': 'http://loinc.org',
       'code': '2708-6',
       'display': 'Smiles per Second'},
      {'system': 'http://loinc.org',
       'code': '59408-5',
       'display': 'Smiles per Second our Device'},
      {'system': 'u

## Get All Hearts Rates Measures for ourpatient

Again just for illustrations, let's check how to retrieve observations for the other metric registred to the patient - heart rate.

In [26]:
get_resource(f"Observation?patient=ourpatient&category=vital-signs&code=8867-4")

{'resourceType': 'Bundle',
 'id': '8235d830-8e2f-11ee-b96a-0242ac180002',
 'type': 'searchset',
 'timestamp': '2023-11-28T20:36:02Z',
 'total': 2,
 'link': [{'relation': 'self',
   'url': 'http://localhost:52773/fhir/r4/Observation?category=vital-signs&code=8867-4&patient=ourpatient'}],
 'entry': [{'fullUrl': 'http://localhost:52773/fhir/r4/Observation/32',
   'resource': {'resourceType': 'Observation',
    'id': '32',
    'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/vitalsigns'],
     'lastUpdated': '2023-11-28T11:40:25Z',
     'versionId': '1'},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"><p><b>Generated Narrative with Details</b></p><p><b>id</b>: heart-rate</p><p><b>meta</b>: </p><p><b>status</b>: final</p><p><b>category</b>: Vital Signs <span>(Details : {http://terminology.hl7.org/CodeSystem/observation-category code \'vital-signs\' = \'Vital Signs\', given as \'Vital Signs\'})</span></p><p><b>code</b>: Heart rate <spa

## References

https://community.intersystems.com/post/home-based-patient-monitoring-fast-track-fhir-story-mit-grand-hack-2022

https://docs.intersystems.com/irisforhealthlatest/csp/docbook/DocBook.UI.Page.cls?KEY=HXFHIR_server_arch_supported